Download the Flickr datasets.

In [20]:
!wget -q https://raw.githubusercontent.com/KuoAiTe/BERT-Trip/main/toy_example/data/traj-Edin.csv -O traj-Edin.csv
!wget -q https://raw.githubusercontent.com/KuoAiTe/BERT-Trip/main/toy_example/data/traj-Glas.csv -O traj-Glas.csv
!wget -q https://raw.githubusercontent.com/KuoAiTe/BERT-Trip/main/toy_example/data/traj-Melb.csv -O traj-Melb.csv
!wget -q https://raw.githubusercontent.com/KuoAiTe/BERT-Trip/main/toy_example/data/traj-Osak.csv -O traj-Osak.csv
!wget -q https://raw.githubusercontent.com/KuoAiTe/BERT-Trip/main/toy_example/data/traj-Toro.csv -O traj-Toro.csv

Import the metric that computes F1 scores including the inputs.

In [27]:
import numpy as np
import pandas as pd

def calc_F1(traj_act, traj_rec, noloop=False):
    '''Compute recall, precision and F1 for recommended trajectories'''
    assert(isinstance(noloop, bool))
    assert(len(traj_act) > 0)
    assert(len(traj_rec) > 0)

    if noloop == True:
        intersize = len(set(traj_act) & set(traj_rec))
    else:
        match_tags = np.zeros(len(traj_act), dtype=bool)
        for poi in traj_rec:
            for j in range(len(traj_act)):
                if match_tags[j] == False and poi == traj_act[j]:
                    match_tags[j] = True
                    break
        intersize = np.nonzero(match_tags)[0].shape[0]

    recall = intersize / len(traj_act)
    precision = intersize / len(traj_rec)
    F1 = 2 * precision * recall / (precision + recall)
    return F1

Import the function that computes the average F1 score from a dataset.

In [24]:
def compute(filepath):
  df = pd.read_csv(filepath)
  # We do not use trajLen < 3 as no prediction can be made given trip origin and trip destination.
  df = df[df['trajLen'] >= 3]
  grouped = df.groupby('trajID')
  f1_scores = []
  for name, traj in grouped:
    traj_length = len(traj)
    expected = traj['poiID'].values
    # Making all the prediction incorrect.
    predicted = np.full(expected.shape, -1)
    # Prior work concatenate the trip origin, prediction, and the trip destination to compute results.
    # Here we followed prior work.
    # prediction = [origin (expected[0]), predictoin, ..., ..., destination(expected[-1])]
    predicted[0] = expected[0]
    predicted[-1] = expected[-1]
    f1_score = calc_F1(expected, predicted)
    f1_scores.append(f1_score)
  f1_scores = np.array(f1_scores)
  return np.mean(f1_scores)

Finally, we compute the results.

In [29]:
datasets = ['Edin', 'Glas', 'Melb', 'Osak', 'Toro']

for dataset in datasets:
  filepath = f'traj-{dataset}.csv'
  score = compute(filepath)
  print(f'Dataset: {dataset} f1_score: {score} with all incorrect predictions.')

Dataset: Edin f1_score: 0.5290504885930754 with all incorrect predictions.
Dataset: Glas f1_score: 0.5966198979591838 with all incorrect predictions.
Dataset: Melb f1_score: 0.5176667977572955 with all incorrect predictions.
Dataset: Osak f1_score: 0.5950354609929079 with all incorrect predictions.
Dataset: Toro f1_score: 0.580805131849908 with all incorrect predictions.


This is why we need to exclude the trip origin and the trip destination in the computation of predictive performance.